In [1]:
pkg_ver = lambda pkg: "{:<20}{:}".format(pkg.__name__,pkg.__version__)

# ROOT
import uproot
print(pkg_ver(uproot))
import ROOT

# Machine Learning
import sklearn
print(pkg_ver(sklearn))
import torch
print(pkg_ver(torch))

# Data science
import scipy
print(pkg_ver(scipy))
import numpy
print(pkg_ver(numpy))
import pandas
print(pkg_ver(pandas))

# Visualizations
import matplotlib
print(pkg_ver(matplotlib))
import matplotlib.pyplot as plt

import tqdm
print(pkg_ver(tqdm))

uproot              4.3.5
Welcome to JupyROOT 6.26/10
sklearn             1.2.0
torch               1.13.1
scipy               1.10.1
numpy               1.24.2
pandas              1.5.3
matplotlib          3.6.2
tqdm                4.62.3


In [2]:
%jsroot

In [3]:
class spectrum:
    
    def __init__(self, TH1D_, iTED_, Crystal_, Configuration_, Window_, Calibration_):  

        self.__TH1D = TH1D_
        self.__iTED = iTED_
        self.__Crystal = Crystal_
        self.__Configuration = Configuration_
        self.__Window = Window_
        self.__Calibration = Calibration_
        
    def __call__(self, ch):
        return numpy.polyval(self.__Calibration[::-1],ch)
    
    def TH1D(self):
        return self.__TH1D
    
    def Calibration(self):
        return self.__Calibration
    
    def iTED(self):
        return self.__iTED
    
    def Crystal(self):
        return self.__Crystal
    
    def Configuration(self):
        return self.__Configuration
    
    def Window(self):
        return self.__Window
    
    def __repr__(self):
        return "iTED: {},Crystal: {},Configuration: {},Window: {}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window
        )
    
    def __str__(self):
        return "{}.{}.{}.{}".format(
            self.__iTED,
            self.__Crystal,
            self.__Configuration,
            self.__Window
        )

In [4]:
iTEDA_cal = pandas.read_csv(
    "../../data/nTOF_March2022/Energy_Calibrations_02_03_2022/Energy_Calibrations_iTEDA.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDB_cal = pandas.read_csv(
    "../../data/nTOF_March2022/Energy_Calibrations_02_03_2022/Energy_Calibrations_iTEDB.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDC_cal = pandas.read_csv(
    "../../data/nTOF_March2022/Energy_Calibrations_02_03_2022/Energy_Calibrations_iTEDC.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTEDD_cal = pandas.read_csv(
    "../../data/nTOF_March2022/Energy_Calibrations_02_03_2022/Energy_Calibrations_iTEDD.dat",
    delim_whitespace=True,
    names=["P0","P1","P2"]
)

iTED_cal = pandas.concat([iTEDA_cal.T, iTEDB_cal.T, iTEDC_cal.T, iTEDD_cal.T], axis=1, keys=['A', 'B', 'C', 'D'])

iTED_cal

A                                                          B  \
            0           1           2          3           4           0   
P0  110.61900  149.122000  191.620000  117.97700  209.411000  137.653000   
P1    1.83959    1.586990    1.904910    1.61805    1.993920    2.077180   
P2    0.00061    0.000873    0.001742    0.00072    0.001584    0.000714   

                                                             C             \
             1           2           3           4           0          1   
P0  194.732000  195.436000  168.661000  180.225000  181.190000  206.42100   
P1    2.466150    1.772110    2.052320    1.953760    1.897540    1.62356   
P2    0.001284    0.001758    0.000866    0.001206    0.001287    0.00163   

                                                 D                         \
             2           3           4           0           1          2   
P0  199.223000  213.161000  195.264000  104.174000  164.759000  150.91700   
P1    1.943850    2.117480    2.084960    2.027700    1.910040    1.83602   
P2    0.001658    0.001264    0.001542    0.000469    0.000965    0.00090   

                            
             3           4  
P0  157.390000  141.421000  
P1    1.977390    1.816370  
P2    0.001071    0.000824

In [5]:
config = [885, 888, 8811]
CW = [100, 200]

ited = ["A","B","C","D"]
crystal = [0,1,2,3,4]

midx = pandas.MultiIndex.from_product(
    [config, CW],
    names=['Configuration', 'Window']
)

mcol = pandas.MultiIndex.from_product(
    [ited, crystal],
    names=['iTED', 'Crystal']
) 

spectra = pandas.DataFrame(index = midx, columns = mcol)

# Access using spectra[iTED,Crystal][Configuration,Window]

spectra

iTED                    A                        B                        C  \
Crystal                 0    1    2    3    4    0    1    2    3    4    0   
Configuration Window                                                          
885           100     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
              200     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
888           100     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
              200     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
8811          100     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   
              200     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   

iTED                                        D                      
Crystal                 1    2    3    4    0    1    2    3    4  
Configuration Window                                               
885           100     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
              200     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
888           100     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
              200     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
8811          100     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
              200     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN

In [6]:
iTEDA = {
    "8811": ROOT.TFile.Open("../../data/2023-02-23/8811/CW100ns/Cs137-iTEDA-A2_D.2023_02_23_T.12_21_13_C.itedABCD_lab_2023.02.22_4.0v_8811_300s.root","READ"),
     "888": ROOT.TFile.Open("../../data/2023-02-23/888/CW100ns/Cs137-iTEDA-A2_D.2023_02_23_T.13_40_46_C.itedABCD_lab_2023.02.22_4.0v_888_300s.root","READ"  )
}

iTEDB = {
    "8811": ROOT.TFile.Open("../../data/2023-03-02/8811/CW100ns/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_10_36_C.itedABCD_lab_2023.02.22_4.0v_8811_300s.root","READ"),
     "888": ROOT.TFile.Open("../../data/2023-03-02/888/CW100ns/Cs137_AfterReconnecting_gum_iTEDB-middle_D.2023_03_02_T.15_22_31_C.itedABCD_lab_2023.02.22_4.0v_888_300s.root","READ"  )
}

iTEDC = {
    "8811": ROOT.TFile.Open("../../data/2023-02-23/8811/CW100ns/Cs137-iTEDC-A1_D.2023_02_23_T.12_41_32_C.itedABCD_lab_2023.02.22_4.0v_8811_300s.root","READ"),
     "888": ROOT.TFile.Open("../../data/2023-02-23/888/CW100ns/Cs137-iTEDC-A1_D.2023_02_23_T.13_26_03_C.itedABCD_lab_2023.02.22_4.0v_888_300s.root","READ"  )
}

iTEDD = {
    "8811": ROOT.TFile.Open("../../data/2023-02-23/8811/CW100ns/Cs137-iTEDD-A1_D.2023_02_23_T.12_48_46_C.itedABCD_lab_2023.02.22_4.0v_8811_300s.root","READ"),
     "888": ROOT.TFile.Open("../../data/2023-02-23/888/CW100ns/Cs137-iTEDD-A1_D.2023_02_23_T.13_03_18_C.itedABCD_lab_2023.02.22_4.0v_888_300s.root","READ"  )
}

In [7]:
for iTED in[0,1,2,3]:
    for Crystal in [0,1,2,3,4]:
        for Configuration in [888,8811]:
            
            spectra[
                ["A","B","C","D"][iTED], Crystal
            ][
                Configuration, 100
            ] = spectrum(
                [iTEDA,iTEDB,iTEDC,iTEDD][iTED][str(Configuration)].Get(
                    "{}_{}_amplitude_spectra;1".format(
                        "SCATTERER" if Crystal==0 else "ABSORBER",
                        ["A","B","C","D"][iTED] if Crystal==0 else "{}_{}".format(["A","B","C","D"][iTED],Crystal)
                    )
                ), 
                ["A","B","C","D"][iTED], 
                Crystal, 
                Configuration, 
                100, 
                iTED_cal[:][["A","B","C","D"][iTED],Crystal]
            )

In [8]:
spectra.dropna(inplace=True)
spectra.applymap(str)

iTED                             A                                            \
Crystal                          0             1             2             3   
Configuration Window                                                           
888           100      A.0.888.100   A.1.888.100   A.2.888.100   A.3.888.100   
8811          100     A.0.8811.100  A.1.8811.100  A.2.8811.100  A.3.8811.100   

iTED                                           B                              \
Crystal                          4             0             1             2   
Configuration Window                                                           
888           100      A.4.888.100   B.0.888.100   B.1.888.100   B.2.888.100   
8811          100     A.4.8811.100  B.0.8811.100  B.1.8811.100  B.2.8811.100   

iTED                                                         C                \
Crystal                          3             4             0             1   
Configuration Window                                                           
888           100      B.3.888.100   B.4.888.100   C.0.888.100   C.1.888.100   
8811          100     B.3.8811.100  B.4.8811.100  C.0.8811.100  C.1.8811.100   

iTED                                                                       D  \
Crystal                          2             3             4             0   
Configuration Window                                                           
888           100      C.2.888.100   C.3.888.100   C.4.888.100   D.0.888.100   
8811          100     C.2.8811.100  C.3.8811.100  C.4.8811.100  D.0.8811.100   

iTED                                                                          
Crystal                          1             2             3             4  
Configuration Window                                                          
888           100      D.1.888.100   D.2.888.100   D.3.888.100   D.4.888.100  
8811          100     D.1.8811.100  D.2.8811.100  D.3.8811.100  D.4.8811.100

In [9]:
def get_resolution(cell):
    
    TH1D = cell.TH1D()
    
    TH1D.GetXaxis().SetRange(TH1D.FindBin(100),TH1D.FindBin(400))
    
    MaxBin   = TH1D.FindBin(TH1D.GetMaximumBin())
    
    ADC_Low  = MaxBin-60
    ADC_High = MaxBin+40

    gaussFit = ROOT.TF1("gaussFit", "pol1(0)+gaus(2)", ADC_Low, ADC_High)
    gaussFit.SetParameters(1000,-1,TH1D.GetMaximum(),MaxBin,10)
    TH1D.Fit(gaussFit,"QR")
    
    sigma = abs(gaussFit.GetParameter(4))
    centroid_ch = gaussFit.GetParameter(3)
    
    x1 = cell(centroid_ch+sigma*numpy.sqrt(2*numpy.log(2)))
    x2 = cell(centroid_ch-sigma*numpy.sqrt(2*numpy.log(2)))
    centroid = cell(centroid_ch)
        
    fwhm = x1-x2
            
    return sigma*numpy.sqrt(2*numpy.log(2))*2/centroid_ch*100, (fwhm/centroid)*100, centroid

In [10]:
def TH1D_draw(cell):
    
    TH1D = cell.TH1D()
    
    canvas = ROOT.TCanvas()
    canvas.cd()
    
    TH1D.SetTitle(repr(cell))
    TH1D.SetStats(False)
    
    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextSize(0.03)
    
    TH1D.Draw("pe")
    
    l1,l2,l3 = get_resolution(cell)
    
    l4 = uproot.open(f"../../data/nTOF_March2022/{cell.Configuration()}/CW100ns/Resolutions_Cs137_CenterScatter_iTED{cell.iTED()}_8.8.{str(cell.Configuration())[2:]}_100ns.root:grResolEnergy;1").values()[1][cell.Crystal()]
    
    latex.DrawText(0.7, 0.8, "R_ch: {:.2f}%".format(l1))
    
    if l4*0.9 < l2 < l4*1.1:
        latex.DrawText(0.7, 0.75, "R_E: {:.2f}%".format(l2))
    else:
        latex.DrawText(0.7, 0.75, "->R_E: {:.2f}%".format(l2))
        
    if 662*(1-l2/100) < l3 < 662*(1+l2/100):
        latex.DrawText(0.7, 0.7, "E: {:.0f}keV".format(l3))
    else:
        latex.DrawText(0.7, 0.7, "->E: {:.0f}keV".format(l3))
        
    latex.DrawText(0.7, 0.65, "R_E(old): {:.2f}%".format(l4))
    latex.DrawText(0.7, 0.6, "Change: {:.2f}%".format(l2-l4))
            
    return canvas

In [11]:
spectra.applymap(lambda x: get_resolution(x)[0]).T.describe()

Configuration,888,8811
Window,100,100
count,20.000000,20.000000
mean,8.384060,9.814858
std,1.506087,2.213196
min,6.573305,6.952911
25%,7.075586,8.021057
50%,8.225662,9.590480
75%,9.645063,11.712712
max,11.434119,13.767949


Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [12]:
spectra.applymap(lambda x: get_resolution(x)[1]).T.describe()

Configuration,888,8811
Window,100,100
count,20.000000,20.000000
mean,7.051488,8.133948
std,1.088098,1.490533
min,5.689949,6.091415
25%,6.113768,6.773125
50%,6.864425,7.963079
75%,7.896744,9.064994
max,9.753728,10.976813


In [13]:
spectra.applymap(lambda x: get_resolution(x)[2]).T.describe()

Configuration,888,8811
Window,100,100
count,20.000000,20.000000
mean,685.584240,670.655993
std,33.159985,39.168601
min,639.367431,624.580485
25%,664.408995,639.257420
50%,675.982817,667.967670
75%,697.379218,686.785620
max,775.907115,783.755354


In [14]:
spectra.applymap(lambda x: get_resolution(x)[1]).style.background_gradient(cmap ='YlOrRd',axis=None)

In [15]:
canvas = spectra.applymap(TH1D_draw)
canvas.applymap(lambda x: type(x).__name__)

iTED                        A                                            B  \
Crystal                     0        1        2        3        4        0   
Configuration Window                                                         
888           100     TCanvas  TCanvas  TCanvas  TCanvas  TCanvas  TCanvas   
8811          100     TCanvas  TCanvas  TCanvas  TCanvas  TCanvas  TCanvas   

iTED                                                            C           \
Crystal                     1        2        3        4        0        1   
Configuration Window                                                         
888           100     TCanvas  TCanvas  TCanvas  TCanvas  TCanvas  TCanvas   
8811          100     TCanvas  TCanvas  TCanvas  TCanvas  TCanvas  TCanvas   

iTED                                                   D                    \
Crystal                     2        3        4        0        1        2   
Configuration Window                                                         
888           100     TCanvas  TCanvas  TCanvas  TCanvas  TCanvas  TCanvas   
8811          100     TCanvas  TCanvas  TCanvas  TCanvas  TCanvas  TCanvas   

iTED                                    
Crystal                     3        4  
Configuration Window                    
888           100     TCanvas  TCanvas  
8811          100     TCanvas  TCanvas

In [16]:
for col in canvas.columns:
    for row in canvas.index:
        canvas.loc[row,col].Draw()